In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaDeltaExample") \
    .config("spark.jars", "/work/spark-sql-kafka-0-10_2.12-3.5.1.jar,/work/kafka-clients-3.7.0.jar") \
    .getOrCreate()


In [2]:
# from pyspark.sql import SparkSession

# spark_dl = SparkSession.builder \
#     .appName("DeltaLakeExample") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:3.1.0,spark-sql-kafka-0-10_2.12-3.5.1") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .getOrCreate()


In [3]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "Kafka:9092") \
    .option("subscribe", "real_time_data_options") \
    .option("startingOffsets", "latest") \
    .load()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [3]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, StructType

# Define the schema of your JSON data
schema = StructType([
    StructField("jsonrpc", StringType()),
    StructField("method", StringType()),
    StructField("params", StructType([
        StructField("channel", StringType()),
        StructField("data", StructType([
            StructField("funding_8h", DoubleType()),
            StructField("current_funding", DoubleType()),
            StructField("estimated_delivery_price", DoubleType()),
            StructField("best_bid_amount", DoubleType()),
            StructField("best_ask_amount", DoubleType()),
            StructField("best_bid_price", DoubleType()),
            StructField("best_ask_price", DoubleType()),
            StructField("interest_value", DoubleType()),
            StructField("open_interest", LongType()),
            StructField("max_price", DoubleType()),
            StructField("min_price", DoubleType()),
            StructField("last_price", DoubleType()),
            StructField("settlement_price", DoubleType()),
            StructField("instrument_name", StringType()),
            StructField("mark_price", DoubleType()),
            StructField("index_price", DoubleType()),
            StructField("stats", StructType([
                StructField("volume_notional", DoubleType()),
                StructField("volume_usd", DoubleType()),
                StructField("volume", DoubleType()),
                StructField("price_change", DoubleType()),
                StructField("low", DoubleType()),
                StructField("high", DoubleType()),
            ])),
            StructField("state", StringType()),
            StructField("timestamp", LongType()),
        ])),
    ])),
])

In [4]:

parsed_df = df.select(from_json(col("value").cast("string"), schema).alias("parsed_value"))

NameError: name 'df' is not defined

In [12]:
from pyspark.sql.functions import col
from pyspark.sql.functions import col, from_unixtime, to_timestamp

flattened_df = parsed_df.select(
    col("parsed_value.jsonrpc"),
    col("parsed_value.method"),
    col("parsed_value.params.channel"),
    col("parsed_value.params.data.funding_8h").alias("funding_8h"),
    col("parsed_value.params.data.current_funding").alias("current_funding"),
    col("parsed_value.params.data.estimated_delivery_price").alias("estimated_delivery_price"),
    col("parsed_value.params.data.best_bid_amount").alias("best_bid_amount"),
    col("parsed_value.params.data.best_ask_amount").alias("best_ask_amount"),
    col("parsed_value.params.data.best_bid_price").alias("best_bid_price"),
    col("parsed_value.params.data.best_ask_price").alias("best_ask_price"),
    col("parsed_value.params.data.interest_value").alias("interest_value"),
    col("parsed_value.params.data.open_interest").alias("open_interest"),
    col("parsed_value.params.data.max_price").alias("max_price"),
    col("parsed_value.params.data.min_price").alias("min_price"),
    col("parsed_value.params.data.last_price").alias("last_price"),
    col("parsed_value.params.data.settlement_price").alias("settlement_price"),
    col("parsed_value.params.data.instrument_name").alias("instrument_name"),
    col("parsed_value.params.data.mark_price").alias("mark_price"),
    col("parsed_value.params.data.index_price").alias("index_price"),
    # col("parsed_value.params.data.stats.volume_notional").alias("volume_notional"),
    # col("parsed_value.params.data.stats.volume_usd").alias("volume_usd"),
    # col("parsed_value.params.data.stats.volume").alias("volume"),
    # col("parsed_value.params.data.stats.price_change").alias("price_change"),
    # col("parsed_value.params.data.stats.low").alias("low"),
    # col("parsed_value.params.data.stats.high").alias("high"),
    # col("parsed_value.params.data.state").alias("state"),
    col("parsed_value.params.data.timestamp").alias("timestamp"),
    to_timestamp(from_unixtime(col("parsed_value.params.data.timestamp") / 1000)).alias("datetimestamp")
)



In [7]:
# Collect and display a small batch of streaming data (for demonstration only)
import time
query = flattened_df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("my_streaming_table") \
    .start()

time.sleep(10)  # Wait a few seconds for data to be processed

# Now you can query the in-memory table as a static DataFrame
display(spark.sql("SELECT * FROM my_streaming_table LIMIT 20").toPandas())

query.stop()  # Don't forget to stop the streaming query

Py4JJavaError: An error occurred while calling o96.start.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catalog.CatalogMana